In [1]:
!pip install requests

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install scrapy

     |████████████████████████████████| 245kB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 3.2MB 166kB/s eta 0:00:01
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=8748 sha256=69cc82a4b40e11f29fa1e239a9dfbfbd8e1041d3ad2ddb6aed81510a53bd4f92
  Stored in directory: /home/nbuser/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=12728 sha256=769e343ca16099910f9d0961a84ae3dc9d703030110ebf6f17b9663157d0cde2
  Stored in directory: /home/nbuser/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego PyDispatcher
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import scrapy
import requests
import json
from datetime import date, timedelta, datetime

In [4]:
def item_json(url):
    '''Gets the item data from the JSON of an item page.
    INPUT: Item page URL
    OUTPUT: JSON data'''

    # Get a response from the URL
    response = scrapy.Selector(text=requests.get(url, timeout=10).text)
    
    # Get tracking data
    track_item_response = response.xpath('//meta[@name="buyma:track_item_json"]/@content').get()
    track_item_json = json.loads(track_item_response)
    # Get main item data
    recent_item_response = response.xpath('//meta[@name="buyma:recent_item_json"]/@content').get()
    recent_item_json = json.loads(recent_item_response)
    # Get access data
    access = response.xpath('//span[@class="ac_count"]/text()').get()
    # Get favourite data
    fav = response.xpath('//span[@class="fav_count"]/text()').get().split('人')[0]
    
    # Combine JSON data and output
    return {'url':url, 'access_count': int(access), 'fav_count': int(fav), **recent_item_json,**track_item_json}

In [5]:
item_json('https://www.buyma.com/item/53799545/')

{'url': 'https://www.buyma.com/item/53799545/',
 'access_count': 603,
 'fav_count': 26,
 'syo_id': 53799545,
 'syo_name': 'CHANEL スニーカー シューズ 靴 CC ホワイト ロゴ 人気 新作 男',
 'syo_img1': 'https://static-buyma-com.akamaized.net/imgdata/item/200427/0053799545/256874963/100.jpg',
 'syo_img_090_1': 'https://static-buyma-com.akamaized.net/imgdata/item/200427/0053799545/256874963/090.jpg',
 'syo_img_210_1': 'https://static-buyma-com.akamaized.net/imgdata/item/200427/0053799545/256874963/210.jpg',
 'syo_url': 'https://www.buyma.com/item/53799545/',
 'tanka_format': '153,720',
 'discount_percentage': None,
 'on_timesale': '',
 'brand_name_eigo': 'CHANEL',
 'brand_url': 'https://www.buyma.com/brand/CHANEL_%E3%82%B7%E3%83%A3%E3%83%8D%E3%83%AB.html',
 'category_id': 3321,
 'category': 'メンズファッション/靴・ブーツ・サンダル',
 'buyer_id': 4880785,
 'brand_id': 158,
 'model_id': 104,
 'season_id': 32,
 'thm_id': 98,
 'kokaidate': '2020-04-27T11:07:53+09:00',
 'yukodate': '2020-07-30T23:59:59+09:00',
 'cate_id1': 1002,
 'cate

In [6]:
buyer_url = 'https://www.buyma.com/buyer/4880785/sales_1.html'
buyer_response = scrapy.Selector(text=requests.get(buyer_url, timeout=10).text)

In [7]:
buyer_table = buyer_response.xpath('//div[@id="buyeritemtable"]')

In [8]:
item_url_extensions = buyer_table.xpath('..//li[@class="buyeritemtable_img"]/a/@href').extract()

In [9]:
item_images = buyer_table.xpath('..//img/@src').extract()

In [10]:
item_names = buyer_table.xpath('..//img/@alt').extract()

In [11]:
sold_amounts_unformatted = buyer_table.xpath('..//li[@class="buyeritemtable_info"]/p[2]/text()').extract()
sold_amounts = [int(i.split('：')[1].split('個')[0]) for i in sold_amounts_unformatted]

In [12]:
sold_dates_unformatted = buyer_table.xpath('..//li[@class="buyeritemtable_info"]/p[3]/text()').extract()
sold_dates = [datetime.strptime(i.split('：')[1], '%Y/%m/%d')  for i in sold_dates_unformatted]

In [13]:
date_threshold = datetime.today() - timedelta(60)

In [14]:
sold_dates[-1] > date_threshold

True

In [15]:
def seller_list(buyer_page_url, previous_days):
    
    items_dict = []
    dt_threshold = datetime.today() - timedelta(previous_days)
    in_time_frame = True
    page_number=1
    buyer_id = buyer_page_url.split('/')[4]
    while True:
        try:
            url = 'https://www.buyma.com/buyer/{}/sales_{}.html'.format(buyer_id,page_number)
            
            response = scrapy.Selector(text=requests.get(url, timeout=10).text)
            buyer_table = response.xpath('//div[@id="buyeritemtable"]')

            # Get item urls
            item_url_extensions = buyer_table.xpath('..//li[@class="buyeritemtable_img"]/a/@href').extract()
            # Get item_images
            item_images = buyer_table.xpath('..//img/@src').extract()
            # Get item_names
            item_names = buyer_table.xpath('..//img/@alt').extract()
            # Get sold amounts
            sold_amounts_unformatted = buyer_table.xpath('..//li[@class="buyeritemtable_info"]/p[2]/text()').extract()
            sold_amounts = [int(i.split('：')[1].split('個')[0]) for i in sold_amounts_unformatted]
            # Get sold dates
            sold_dates_unformatted = buyer_table.xpath('..//li[@class="buyeritemtable_info"]/p[3]/text()').extract()
            sold_dates = [datetime.strptime(i.split('：')[1], '%Y/%m/%d')  for i in sold_dates_unformatted]

            keys = ['url_ext', 'img', 'item_name','sold_amount', 'sold_date']

            # Combine to a dictionary
            items_dict+= [dict(zip(keys,[item_url_extensions[i],
                              item_images[i],
                              item_names[i],
                              sold_amounts[i],
                              sold_dates[i]])) 
                          for i in range(len(item_url_extensions))]
            print('Buyer page:', page_number)
            print('Last date sold:', sold_dates[-1])
            
            # Loop check
            if sold_dates[-1] > dt_threshold:
                page_number+=1
            else:
                print('end')
                break
                
        except:
            print('No more item pages to get in time frame')
            break

        
    return items_dict

In [16]:
sold_dates[-1]

datetime.datetime(2020, 6, 3, 0, 0)

In [17]:
date_threshold = datetime.today() - timedelta(30)
sold_dates[-1] > date_threshold

False

In [18]:
seller_list('https://www.buyma.com/buyer/4880785/sales_1.html', 60)


Buyer page: 1
Last date sold: 2020-06-03 00:00:00
Buyer page: 2
Last date sold: 2020-02-24 00:00:00
end


[{'url_ext': '/item/46113739/',
  'img': 'https://static-buyma-com.akamaized.net/imgdata/item/190808/0046113739/209733446/210.jpg',
  'item_name': 'Dior バッグ ショルダー ブラック オブリーク ロゴ 直営店 19AW',
  'sold_amount': 1,
  'sold_date': datetime.datetime(2020, 7, 17, 0, 0)},
 {'url_ext': '/item/43629712/',
  'img': 'https://static-buyma-com.akamaized.net/imgdata/item/190510/0043629712/195083141/210.jpg',
  'item_name': 'オプション/MY CLOSET*',
  'sold_amount': 1,
  'sold_date': datetime.datetime(2020, 7, 14, 0, 0)},
 {'url_ext': '/item/44765205/',
  'img': 'https://static-buyma-com.akamaized.net/imgdata/item/190620/0044765205/201770895/210.jpg',
  'item_name': 'Dior サンダル ネオプレン カーフ ブラック CD ロゴ メンズ 19AW',
  'sold_amount': 1,
  'sold_date': datetime.datetime(2020, 7, 10, 0, 0)},
 {'url_ext': '/item/45827554/',
  'img': 'https://static-buyma-com.akamaized.net/imgdata/item/190729/0045827554/269034514/210.jpg',
  'item_name': 'CHANEL ポーチ カード コイン ホルダー マト CC ラム ブラック',
  'sold_amount': 1,
  'sold_date': datetime.d